In [1]:
from multicall import Call

from mainnet_launch.data_fetching.get_state_by_block import get_raw_state_by_blocks, build_blocks_to_use
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import streamlit as st
from multicall import Call
from web3 import Web3


from mainnet_launch.constants import (
    AutopoolConstants,
    WETH,
    ROOT_PRICE_ORACLE,
    ChainData,
    ALL_AUTOPOOLS,
    BASE_ETH,
    eth_client,
    ETH_CHAIN,
    AUTO_ETH,
)

from mainnet_launch.data_fetching.get_state_by_block import (
    get_raw_state_by_blocks,
    safe_normalize_with_bool_success,
    get_state_by_one_block,
    identity_with_bool_success,
)

from mainnet_launch.data_fetching.get_events import fetch_events

from mainnet_launch.abis import (
    AUTOPOOL_ETH_STRATEGY_ABI,
    ERC_20_ABI,
    BALANCER_AURA_DESTINATION_VAULT_ABI,
    ROOT_PRICE_ORACLE_ABI,
)
from mainnet_launch.destinations import get_destination_details, DestinationDetails
from mainnet_launch.data_fetching.add_info_to_dataframes import (
    add_timestamp_to_df_with_block_column,
    add_transaction_gas_info_to_df_with_tx_hash,
)

from mainnet_launch.database.should_update_database import should_update_table
from mainnet_launch.pages.asset_discounts.fetch_and_render_asset_discounts import _fetch_lst_calc_addresses_df


blocks = build_blocks_to_use(AUTO_ETH.chain)
len(blocks)

2025-01-22 17:57:28.533 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-01-22 17:57:28.614 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-22 17:57:28.775 
  command:

    streamlit run /home/parker/Documents/Tokemak/v2-rebalance-dashboard/.venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-01-22 17:57:28.776 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-22 17:57:28.776 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-22 17:57:28.777 No runtime found, using MemoryCacheStorageManager
2025-01-22 17:57:29.277 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-22 17:57:29.278 Thread 'Thread-4': missing Script

513

In [2]:
# get all the destination vaults

a_vault = "0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B"


underlying_reserves_call = Call(
    a_vault,
    ["underlyingReserves()(address[],uint256[])"],
    [
        ("underlyingReserves_tokens", None),  # Placeholder for parsing tokens
        ("underlyingReserves_amounts", None),  # Placeholder for parsing amounts
    ],
)

get_state_by_one_block([underlying_reserves_call], eth_client.eth.block_number, ETH_CHAIN)

{'underlyingReserves_tokens': ('0x04c154b66cb340f3ae24111cc767e0184ed00cc6',
  '0x88794c65550deb6b4087b7552ecf295113794410',
  '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'),
 'underlyingReserves_amounts': (7949892059228140226882,
  2596148429267417491710537265265938,
  2954771257518689298733)}

In [3]:
details = get_destination_details(AUTO_ETH)

details

[Destination Details:
  vaultAddress	: 0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
  exchangeName	: tokemak
  dexPool	: None
  lpTokenAddress	: 0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56
  lpTokenSymbol	: None
  lpTokenName	: None
  autopool	: {'name': 'autoETH', 'autopool_eth_addr': '0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56', 'autopool_eth_strategy_addr': '0xf5f6addB08c5e6091e5FdEc7326B21bEEd942235', 'solver_rebalance_plans_bucket': 'v2-gen3-eth-mainnet-solve-v2gen3ethmainnetsolverau-7ju0ra8aoznh', 'chain': {'name': 'eth', 'block_autopool_first_deployed': 20752910, 'approx_seconds_per_block': 12.0, 'chain_id': 1}}
  vault_name	: autoETH (tokemak),
 Destination Details:
  vaultAddress	: 0x1E581AD9ef141C3037991806383B7990c1f68312
  exchangeName	: balancer
  dexPool	: 0x1E19CF2D73a72Ef1332C882F20534B6519Be0276
  lpTokenAddress	: 0x1E19CF2D73a72Ef1332C882F20534B6519Be0276
  lpTokenSymbol	: B-rETH-STABLE
  lpTokenName	: Balancer rETH Stable Pool
  autopool	: {'name': 'autoETH', 'autopool_

In [4]:
set([d.vault_name for d in details])

{'B-rETH-STABLE (balancer)',
 'ECLP-wstETH-cbETH (balancer)',
 'ECLP-wstETH-wETH (balancer)',
 'ETHx/wstETH (balancer)',
 'autoETH (tokemak)',
 'ethx-f (curve)',
 'frxeth-ng-f (curve)',
 'osETH-rETH (curve)',
 'osETH/wETH-BPT (balancer)',
 'pxETH/wETH (balancer)',
 'pxethweth (curve)',
 'pxsteth (curve)',
 'wETHrETH (curve)',
 'wstETH-WETH-BPT (balancer)'}

In [5]:
def same_normalize_with_bool_success_list(success, values):
    if success:
        return tuple([int(val) / 1e18 for val in values])


def make_calls(dest: DestinationDetails):
    unique_id = f"{dest.vault_name} {dest.vaultAddress}"
    underlying_reserves_call = Call(
        dest.vaultAddress,
        ["underlyingReserves()(address[],uint256[])"],
        [
            (f"{unique_id}_underlyingReserves_tokens", None),
            (f"{unique_id}_underlyingReserves_amounts", same_normalize_with_bool_success_list),
        ],
    )
    underlyingTotalSupply_call = Call(
        dest.vaultAddress,
        ["underlyingTotalSupply()(uint256)"],
        [
            (f"{unique_id}_total_supply", safe_normalize_with_bool_success),
        ],
    )

    balance_of_call = Call(
        dest.vaultAddress,
        ["balanceOf(address)(uint256)", dest.autopool.autopool_eth_addr],
        [
            (f"{unique_id}_autopool_lp_tokens", safe_normalize_with_bool_success),  # Extract the balance (uint256)
        ],
    )
    return [underlying_reserves_call, underlyingTotalSupply_call, balance_of_call], unique_id


calls, id = make_calls(details[-1])

all_calls = []
all_ids = []
id_to_dest = {}
for dest in details:
    if dest.autopool not in [ALL_AUTOPOOLS]:
        calls, unique_id = make_calls(dest)
        all_calls.extend(calls)
        all_ids.append(unique_id)
        id_to_dest[unique_id] = dest


row = get_state_by_one_block(all_calls, eth_client.eth.block_number, ETH_CHAIN)

In [6]:
df = get_raw_state_by_blocks(all_calls, blocks, ETH_CHAIN)
df

,autoETH (tokemak) 0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56_underlyingReserves_tokens,autoETH (tokemak) 0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56_underlyingReserves_amounts,autoETH (tokemak) 0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56_total_supply,autoETH (tokemak) 0x0A2b94F6871c1D7A32Fe58E1ab5e6deA2f114E56_autopool_lp_tokens,B-rETH-STABLE (balancer) 0x1E581AD9ef141C3037991806383B7990c1f68312_underlyingReserves_tokens,B-rETH-STABLE (balancer) 0x1E581AD9ef141C3037991806383B7990c1f68312_underlyingReserves_amounts,B-rETH-STABLE (balancer) 0x1E581AD9ef141C3037991806383B7990c1f68312_total_supply,B-rETH-STABLE (balancer) 0x1E581AD9ef141C3037991806383B7990c1f68312_autopool_lp_tokens,osETH/wETH-BPT (balancer) 0x013D82483C968cdbB4a8384754f5a5028b85dB6d_underlyingReserves_tokens,osETH/wETH-BPT (balancer) 0x013D82483C968cdbB4a8384754f5a5028b85dB6d_underlyingReserves_amounts,...,ethx-f (curve) 0x1Ea622fa030e4a78F4CC2f305dd3c08DA3F08573_total_supply,ethx-f (curve) 0x1Ea622fa030e4a78F4CC2f305dd3c08DA3F08573_autopool_lp_tokens,ETHx/wstETH (balancer) 0xdfE3fA7027E84f59b266459C567278C79fe86f0C_underlyingReserves_tokens,ETHx/wstETH (balancer) 0xdfE3fA7027E84f59b266459C567278C79fe86f0C_underlyingReserves_amounts,ETHx/wstETH (balancer) 0xdfE3fA7027E84f59b266459C567278C79fe86f0C_total_supply,ETHx/wstETH (balancer) 0xdfE3fA7027E84f59b266459C567278C79fe86f0C_autopool_lp_tokens,pxETH/wETH (balancer) 0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B_underlyingReserves_tokens,pxETH/wETH (balancer) 0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B_underlyingReserves_amounts,pxETH/wETH (balancer) 0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B_total_supply,pxETH/wETH (balancer) 0xc4Eb861e7b66f593482a3D7E8adc314f6eEDA30B_autopool_lp_tokens
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-15 02:04:47+00:00,None,None,None,0.000000,"(0xae78736cd615f374d3085123a210448e74fc6393, 0...","(5853.3132088474695, 7013.31309808448)",13076.818603,0.0,"(0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2, 0...","(11973.086949944378, 2596148429267418.5, 7888....",...,NaN,NaN,None,None,NaN,NaN,None,None,NaN,NaN
2024-09-15 08:06:23+00:00,None,None,None,0.000000,"(0xae78736cd615f374d3085123a210448e74fc6393, 0...","(5848.638308837697, 7051.931502912554)",13109.227832,0.0,"(0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2, 0...","(11973.086949944378, 2596148429267418.5, 7888....",...,NaN,NaN,None,None,NaN,NaN,None,None,NaN,NaN
2024-09-15 14:07:35+00:00,None,None,None,0.000000,"(0xae78736cd615f374d3085123a210448e74fc6393, 0...","(5860.6979428119175, 7088.448747262166)",13157.451200,0.0,"(0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2, 0...","(11974.016949944378, 2596148429267418.5, 7887....",...,NaN,NaN,None,None,NaN,NaN,None,None,NaN,NaN
2024-09-15 20:08:35+00:00,None,None,None,0.000000,"(0xae78736cd615f374d3085123a210448e74fc6393, 0...","(5870.73372025148, 7077.235574828276)",13157.451200,0.0,"(0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2, 0...","(11974.016949944378, 2596148429267418.5, 7887....",...,NaN,NaN,None,None,NaN,NaN,None,None,NaN,NaN
2024-09-16 02:09:35+00:00,None,None,None,0.000000,"(0xae78736cd615f374d3085123a210448e74fc6393, 0...","(5870.573008604739, 7077.415279490292)",13157.451200,0.0,"(0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2, 0...","(11972.633170485222, 2596148429267418.5, 7889....",...,NaN,NaN,None,None,NaN,NaN,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-20 17:51:11+00:00,None,None,None,0.000000,"(0xae78736cd615f374d3085123a210448e74fc6393, 0...","(7560.8337934343745, 5525.204779329342)",13499.714063,0.0,"(0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2, 0...","(7738.734561186601, 2596148429267420.0, 9917.1...",...,1891.362170,1691.082983,"(0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0, 0...","(176.20351068849396, 148.0163424917768, 259614...",358.468798,102.867372,"(0x04c154b66cb340f3ae24111cc767e0184ed00cc6, 0...","(8238.864021649004, 2596148429267417.5, 3244.0...",11449.781006,4815.545864
2025-01-20 23:52:47+00:00,None,None,None,0.235410

In [7]:
def _extract_quantity_of_assets(row: dict):
    # reutrns a dictionary of token: quantity of that token the vault has ownership over
    quantity_of_assets = {}

    for unique_id, dest in id_to_dest.items():
        lp_total_supply = row[f"{unique_id}_total_supply"]
        autopool_lp_tokens = row[f"{unique_id}_autopool_lp_tokens"]
        if isinstance(lp_total_supply, float) and isinstance(autopool_lp_tokens, float):
            if lp_total_supply > 0 and autopool_lp_tokens > 0:

                portion_ownership_of_pool = autopool_lp_tokens / lp_total_supply

                for token_addr, amount in zip(
                    row[f"{unique_id}_underlyingReserves_tokens"], row[f"{unique_id}_underlyingReserves_amounts"]
                ):
                    if token_addr.lower() != dest.lpTokenAddress.lower():
                        # for balancer stable pools skip the lp token
                        if token_addr not in quantity_of_assets:

                            quantity_of_assets[token_addr] = portion_ownership_of_pool * amount
                        else:
                            quantity_of_assets[token_addr] += portion_ownership_of_pool * amount
    return quantity_of_assets


assets_df = pd.DataFrame.from_records(df.apply(lambda row: _extract_quantity_of_assets(row), axis=1))
assets_df

/tmp/ipykernel_58192/3544499653.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  assets_df = pd.DataFrame.from_records(df.apply(lambda row: _extract_quantity_of_assets(row), axis=1))


,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x04c154b66cb340f3ae24111cc767e0184ed00cc6,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,0xa35b1b31ce002fbf2058d22f30f95d405200a15b,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,0xae78736cd615f374d3085123a210448e74fc6393,0xf1c9acdc66974dfb6decb12aa385b9cd01190e38
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
508,2963.821463,8476.580046,1684.914259,1308.224841,50.563932,NaN,NaN
509,2894.351081,8474.370380,1727.585692,1312.065118,45.545589,NaN,NaN
510,2866.464583,8500.436670,1727.585692,1313.870125,45.545589,NaN,NaN
511,2949.752096,8615.232314,1561.357092,1283.564554,45.544589,NaN,NaN


In [ ]:
# fetch the safe price
# fetch the backing price

# compare them over time on the asset allocation tab

In [8]:
import plotly.io as pio

pio.templates.default = None
px.bar(assets_df)